## Anime Merge

In [1]:
iguals = ['anime_id', 'title', 'score', 'episodes', 'start_date', 'end_date', 'members', 'favorites', 'sfw', 'approved', 'real_start_date', \
    'real_end_date', 'genres', 'themes', 'demographics', 'studios', 'main_picture', 'title_english', 'title_japanese']

mal_millor = ['scored_by', 'episode_duration', 'start_year', 'start_season', 'broadcast_time', 'synopsis', 'title_synonyms']

mal_extra = ['total_duration', 'created_at', 'updated_at']

formatted_strings = ['type', 'status', 'source', 'rating', 'broadcast_day']

jikan_extra = ['producers', 'licensors', 'background', 'url', 'trailer_url']

In [2]:
import pandas as pd
import ast

mal = pd.read_csv('data/anime_mal.csv')
jikan = pd.read_csv('data/anime_jikan.csv')

anime = pd.merge(mal[iguals + mal_millor + mal_extra + formatted_strings], jikan[['anime_id'] + jikan_extra], on='anime_id')

order = ['anime_id', 'title', 'type', 'score', 'scored_by', 'status', 'episodes', 'start_date', 'end_date', 'source',
         'members', 'favorites', 'episode_duration', 'total_duration', 'rating', 'sfw', 'approved', 'created_at', 'updated_at',
         'start_year', 'start_season', 'real_start_date', 'real_end_date', 'broadcast_day', 'broadcast_time',
         'genres', 'themes', 'demographics', 'studios', 'producers', 'licensors', 'synopsis', 'background',
         'main_picture', 'url', 'trailer_url', 'title_english', 'title_japanese', 'title_synonyms']

anime = anime[order]

for col in ['start_date', 'end_date', 'created_at', 'updated_at']:
    anime[col] = pd.to_datetime(anime[col])

for col in ['episodes', 'start_year']:
    anime[col] = anime[col].astype('Int64')

for col in ['genres', 'themes', 'demographics', 'studios', 'producers', 'licensors', 'title_synonyms']:
    anime[col] = anime[col].apply(ast.literal_eval)

anime['broadcast_time'] = pd.to_datetime(anime['broadcast_time']).dt.time

anime['episode_duration'] = pd.to_timedelta(anime['episode_duration'])

print(anime.shape)

anime.to_csv('data/anime.csv', index=False)

pd.options.display.max_columns = None
anime.head(1)

(24165, 39)


,anime_id,title,type,score,scored_by,status,episodes,start_date,end_date,source,members,favorites,episode_duration,total_duration,rating,sfw,approved,created_at,updated_at,start_year,start_season,real_start_date,real_end_date,broadcast_day,broadcast_time,genres,themes,demographics,studios,producers,licensors,synopsis,background,main_picture,url,trailer_url,title_english,title_japanese,title_synonyms
0,5114,Fullmetal Alchemist: Brotherhood,tv,9.13,1871705,finished_airing,64,2009-04-05,2010-07-04,manga,2932347,204645,0 days 00:24:20,1 days 01:57:20,r,True,True,2008-08-21 03:35:22+00:00,2022-04-18 05:06:13+00:00,2009,spring,2009-04-05,2010-07-04,sunday,17:00:00,"[Action, Adventure, Drama, Fantasy]",[Military],[Shounen],[Bones],"[Aniplex, Square Enix, Mainichi Broadcasting S...","[Funimation, Aniplex of America]",After a horrific alchemy experiment goes wrong...,NaN,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,https://www.youtube.com/watch?v=--IcmZkvL0Q,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,[Hagane no Renkinjutsushi: Fullmetal Alchemist...


## Load Data

In [3]:
import pandas as pd
import ast

anime = pd.read_csv('data/anime.csv')

for col in ['start_date', 'end_date', 'created_at', 'updated_at']:
    anime[col] = pd.to_datetime(anime[col])

for col in ['episodes', 'start_year']:
    anime[col] = anime[col].astype('Int64')

for col in ['genres', 'themes', 'demographics', 'studios', 'producers', 'licensors', 'title_synonyms']:
    anime[col] = anime[col].apply(ast.literal_eval)

anime['broadcast_time'] = pd.to_datetime(anime['broadcast_time']).dt.time

anime['episode_duration'] = pd.to_timedelta(anime['episode_duration'])

pd.options.display.max_columns = None
anime.head(1)

,anime_id,title,type,score,scored_by,status,episodes,start_date,end_date,source,members,favorites,episode_duration,total_duration,rating,sfw,approved,created_at,updated_at,start_year,start_season,real_start_date,real_end_date,broadcast_day,broadcast_time,genres,themes,demographics,studios,producers,licensors,synopsis,background,main_picture,url,trailer_url,title_english,title_japanese,title_synonyms
0,5114,Fullmetal Alchemist: Brotherhood,tv,9.13,1871705,finished_airing,64,2009-04-05,2010-07-04,manga,2932347,204645,0 days 00:24:20,1 days 01:57:20,r,True,True,2008-08-21 03:35:22+00:00,2022-04-18 05:06:13+00:00,2009,spring,2009-04-05,2010-07-04,sunday,17:00:00,"[Action, Adventure, Drama, Fantasy]",[Military],[Shounen],[Bones],"[Aniplex, Square Enix, Mainichi Broadcasting S...","[Funimation, Aniplex of America]",After a horrific alchemy experiment goes wrong...,NaN,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,https://www.youtube.com/watch?v=--IcmZkvL0Q,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,[Hagane no Renkinjutsushi: Fullmetal Alchemist...


## Differences

In [7]:
anime_all = pd.merge(mal, jikan, on='anime_id', how='outer', indicator=True)

anime_all._merge.value_counts()

both          24165
right_only      530
left_only         0
Name: _merge, dtype: int64

In [24]:
# Borrats de MyAnimeList
deleted = jikan[jikan.anime_id.isin(anime_all[anime_all['_merge']=='right_only'].anime_id)]

deleted.shape[0]

530

In [25]:
# Completament iguals
compl_iguals = pd.merge(mal[iguals], jikan[iguals])

compl_iguals.shape[0]

8844

In [19]:
# Iguals
changing = ['score', 'scored_by', 'members', 'favorites']
iguals_not_changing = [col for col in iguals if col not in changing]

df_iguals = pd.merge(mal[iguals_not_changing], jikan[iguals_not_changing])
df_iguals.shape[0]

24095

In [35]:
# Changed

anime_both = pd.merge(mal[iguals_not_changing], jikan[iguals_not_changing], how='outer', indicator=True)

changed = anime_both[~anime_both.anime_id.isin(deleted.anime_id) & (anime_both['_merge']!='both')] \
    .sort_values(['anime_id', '_merge']).drop(columns=['_merge'])

changed.shape[0] // 2

70

In [36]:
import collections as c

changes = c.Counter()

for i in range(changed.shape[0] // 2):
    changed_anime = changed[2*i:2*(i+1)]
    first, second = changed_anime.iloc[0], changed_anime.iloc[1]
    is_diff_cols = (first!=second) & ~(first.isna() & second.isna())
    diff_cols = is_diff_cols[is_diff_cols].index
    for col in diff_cols:
        changes[col] += 1

In [37]:
changes.most_common()

[('studios', 23),
 ('main_picture', 18),
 ('end_date', 15),
 ('real_end_date', 15),
 ('start_date', 14),
 ('real_start_date', 14),
 ('title_japanese', 10),
 ('approved', 6),
 ('episodes', 5),
 ('title', 3),
 ('genres', 3),
 ('title_english', 1)]

In [33]:
changed.head(2)

,anime_id,title,episodes,start_date,end_date,sfw,approved,real_start_date,real_end_date,genres,themes,demographics,studios,main_picture,title_english,title_japanese,_merge
7786,23,Ring ni Kakero 1,12.0,2004-10-06,2004-12-15,True,True,2004-10-06,2004-12-15,"['Action', 'Sports']",['Combat Sports'],['Shounen'],['Toei Animation'],https://cdn.myanimelist.net/images/anime/1146/...,NaN,リングにかけろ１,left_only
24193,23,Ring ni Kakero 1,12.0,2004-10-06,2004-12-15,True,True,2004-10-06,2004-12-15,"['Action', 'Sports']",['Combat Sports'],['Shounen'],['Toei Animation'],https://cdn.myanimelist.net/images/anime/5/534...,NaN,リングにかけろ１,right_only


## Others

In [32]:
# Formatted or unformatted????
formatted_strings = ['type', 'status', 'source', 'rating', 'broadcast_day']

anime_all.groupby(['status_x', 'status_y']).size()

status_x          status_y        
currently_airing  Currently Airing      385
                  Not yet aired           2
finished_airing   Currently Airing        6
                  Finished Airing     23156
not_yet_aired     Not yet aired         616
dtype: int64